# 2-Coding: MELTS liquid model with zirconium

# 2-Coding: MELTS liquid model with zirconium

**Model formulation and code generation notebook.**  
**Run Endmembers-MELTS notebook first!**

Proposed associated solution model:

| Components:                                | Species:                                    | Reaction:                                                                                                                              | Variable: | Component: | Species X: |
|:-------------|:----------|:-----------|:-----------|:------------|:------------|
| SiO<sub>2</sub>                            | SiO<sub>2</sub>                             |                                                                                                                                        |           | $n_1$      | $X_1$      |
| TiO<sub>2</sub>                            | TiO<sub>2</sub>                             |                                                                                                                                        | $r_1$     | $n_2$      | $X_2$      |
| Al<sub>2</sub>O<sub>3</sub>                | Al<sub>2</sub>O<sub>3</sub>                 |                                                                                                                                        | $r_2$     | $n_3$      | $X_3$      |
| Fe<sub>2</sub>O<sub>3</sub>                | Fe<sub>2</sub>O<sub>3</sub>                 |                                                                                                                                        | $r_3$     | $n_4$      | $X_4$      |
| MgCr<sub>2</sub>O<sub>4</sub>              | MgCr<sub>2</sub>O<sub>4</sub>               |                                                                                                                                        | $r_4$     | $n_5$      | $X_5$      |
| Fe<sub>2</sub>SiO<sub>4</sub>              | Fe<sub>2</sub>SiO<sub>4</sub>               |                                                                                                                                        | $r_5$     | $n_6$      | $X_6$      |
| MnSi<sub>1/2</sub>O<sub>2</sub>            | MnSi<sub>1/2</sub>O<sub>2</sub>             |                                                                                                                                        | $r_6$     | $n_7$      | $X_7$      |
| Mg<sub>2</sub>SiO<sub>4</sub>              | Mg<sub>2</sub>SiO<sub>4</sub>               |                                                                                                                                        | $r_7$     | $n_8$      | $X_8$      |
| NiSi<sub>1/2</sub>O<sub>2</sub>            | NiSi<sub>1/2</sub>O<sub>2</sub>             |                                                                                                                                        | $r_8$     | $n_9$      | $X_9$      |
| CoSi<sub>1/2</sub>O<sub>2</sub>            | CoSi<sub>1/2</sub>O<sub>2</sub>             |                                                                                                                                        | $r_9$     | $n_{10}$   | $X_{10}$   |
| CaSiO<sub>3</sub>                          | CaSiO<sub>3</sub>                           |                                                                                                                                        | $r_{10}$  | $n_{11}$   | $X_{11}$   |
| Na<sub>2</sub>SiO<sub>3</sub>              | Na<sub>2</sub>SiO<sub>3</sub>               |                                                                                                                                        | $r_{11}$  | $n_{12}$   | $X_{12}$   |
| KAlSiO<sub>4</sub>                         | KAlSiO<sub>4</sub>                          |                                                                                                                                        | $r_{12}$  | $n_{13}$   | $X_{13}$   |
| Ca<sub>3</sub>(PO<sub>4</sub>)<sub>2</sub> | Ca<sub>3</sub>(PO<sub>4</sub>)<sub>2</sub>  |                                                                                                                                        | $r_{13}$  | $n_{14}$   | $X_{14}$   |
| H<sub>2</sub>O                             | H<sub>2</sub>O                              |                                                                                                                                        | $r_{14}$  | $n_{15}$   | $X_{15}$   |
| CO<sub>2</sub>                             | CO<sub>2</sub>                              |                                                                                                                                        | $r_{15}$  | $n_{16}$   | $X_{16}$   |
| ZrSiO<sub>4</sub>                          | ZrSiO<sub>4</sub>                           |                                                                                                                                        | $r_{16}$  | $n_{17}$   | $X_{17}$   |
|                                            | CaCO<sub>3</sub>                            | CO<sub>2</sub> + CaSiO<sub>3</sub> = CaCO<sub>3</sub> + SiO<sub>2</sub>                                                                | $s_1$     |            | $X_{18}$   |
|                                            | Na<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub> | ZrSiO<sub>4</sub> + 2Na<sub>2</sub>SiO<sub>3</sub> = Na<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub> + SiO<sub>2</sub>                     | $s_2$     |            | $X_{19}$   |
|                                            | K<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub>  | ZrSiO<sub>4</sub> + 4KAlSiO<sub>4</sub> = K<sub>4</sub>ZrSi<sub>2</sub>O<sub>8</sub> + 2Al<sub>2</sub>O<sub>3</sub> + 3SiO<sub>2</sub> | $s_3$     |            | $X_{20}$   |

In [ ]:
import pandas as pd
import numpy as np
import sympy as sym
sym.init_printing()
from thermoengine import core, coder
from thermoengine.model import Database
db = Database()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

-   Number of components = 17
-   Number of species = 20
-   Number of “ordering parameters” (i.e., dependent species) = 3

In [ ]:
nc = 17
nw = 20
ns = 3

## Create solution model

In [ ]:
model = coder.ComplexSolnModel(nc=nc, ns=ns, nw=nw)

-   *n* is a vector of moles of solution components
-   *nT* is a sum of all *n*
-   *s* is a vector of *moles* of dependemt species

In [ ]:
n = model.n
nT = model.nT
s = model.s

Set up ancillary compositional variables - *X* is a vector of *moles* of all species (components + dependent species in solution

Note that the zeroth index of *X* is ignored

In [ ]:
X = [sym.S.Zero]
for i in range(1,nw+1):
    X.append(sym.symbols('X_'+str(i)))

Moles of elements are accumulated in the dictionary *elm*. See table above.

In [ ]:
elm = {}

In [ ]:
O_n = 2*n[0]+2*n[1]+3*n[2]+3*n[3]+4*n[4]+4*n[5]+2*n[6]+4*n[7]+2*n[8]+2*n[9]+3*n[10]+3*n[11]+4*n[12]+8*n[13]+n[14]+2*n[15]+4*n[16]
O_X = 2*X[1]+2*X[2]+3*X[3]+3*X[4]+4*X[5]+4*X[6]+2*X[7]+4*X[8]+2*X[9]+2*X[10]+3*X[11]+3*X[12]+4*X[13]+8*X[14]+X[15] \
    + 2*X[16]+4*X[17]+3*X[18]+8*X[19]+8*X[20]
eqn_O = O_n - O_X
elm[' O'] = O_n

In [ ]:
Si_n = n[0]+n[5]+n[6]/2+n[7]+n[8]/2+n[9]/2+n[10]+n[11]+n[12]+n[16]
Si_X = X[1]+X[6]+X[7]/2+X[8]+X[9]/2+X[10]/2+X[11]+X[12]+X[13]+X[17]+2*X[19]+2*X[20]
eqn_Si = Si_n - Si_X
elm['Si'] = Si_n

In [ ]:
Ti_n = n[1]
Ti_X = X[2]
eqn_Ti = Ti_n - Ti_X
elm['Ti'] = Ti_n

In [ ]:
Al_n = 2*n[2] + n[12]
Al_X = 2*X[3] + X[13]
eqn_Al = Al_n - Al_X
elm['Al'] = Al_n

In [ ]:
Fe_n = 2*n[3] + 2*n[5]
Fe_X = 2*X[4] + 2*X[6]
eqn_Fe = Fe_n - Fe_X
elm['Fe'] = Fe_n

In [ ]:
Cr_n = 2*n[4]
Cr_X = 2*X[5]
eqn_Cr = Cr_n - Cr_X
elm['Cr'] = Cr_n

In [ ]:
Mn_n = n[6]
Mn_X = X[7]
eqn_Mn = Mn_n - Mn_X
elm['Mn'] = Mn_n

In [ ]:
Mg_n = 2*n[7]
Mg_X = 2*X[8]
eqn_Mg = Mg_n - Mg_X
elm['Mg'] = Mg_n

In [ ]:
Ni_n = n[8]
Ni_X = X[9]
eqn_Ni = Ni_n - Ni_X
elm['Ni'] = Ni_n

In [ ]:
Co_n = n[9]
Co_X = X[10]
eqn_Co = Co_n - Co_X
elm['Co'] = Co_n

In [ ]:
Ca_n = n[10] + 3*n[13]
Ca_X = X[11] + 3*X[14] + X[18]
eqn_Ca = Ca_n - Ca_X
elm['Ca'] = Ca_n

In [ ]:
Na_n = 2*n[11]
Na_X = 2*X[12] + 4*X[19]
eqn_Na = Na_n - Na_X
elm['Na'] = Na_n

In [ ]:
K_n = n[12]
K_X = X[13] + 4*X[20]
eqn_K = K_n - K_X
elm[' K'] = K_n

In [ ]:
P_n = 2*n[13]
P_X = 2*X[14]
eqn_P = P_n - P_X
elm[' P'] = P_n

In [ ]:
H_n = 2*n[14]
H_X = 2*X[15]
eqn_H = H_n - H_X
elm[' H'] = H_n

In [ ]:
C_n = n[15]
C_X = X[16] + X[18]
eqn_C = C_n - C_X
elm[' C'] = C_n

In [ ]:
Zr_n = n[16]
Zr_X = X[17] + X[19] + X[20] 
eqn_Zr = Zr_n - Zr_X
elm['Zr'] = Zr_n

Solve this system for species mole numbers

In [ ]:
system = [eqn_O, eqn_Si, eqn_Ti, eqn_Al, eqn_Fe, eqn_Cr, eqn_Mn, eqn_Mg, eqn_Ni, eqn_Co, eqn_Ca, eqn_Na, eqn_K, eqn_P, eqn_H, eqn_C, eqn_Zr,
         s[0]-X[18], s[1]-X[19], s[2]-X[20]]
X_soln = list(sym.linsolve(system, X[1:nw+1]).args[0])

Compute the total number of moles of all species, *XT*. Note rhat this quantity is not necessarily equal to *nT*.

In [ ]:
XT = sym.S.Zero
for i,x in enumerate(X_soln):
    print ('X[',i+1,'] = ', end='') 
    sym.pprint (x)
    XT += x
XT

Retrieve model symbols for the temperature, pressure, and standard state chemical potentials - $T$ is temperature in $K$ - $P$ is pressure in $bars$ - $\mu^o$ in Joules

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
mu = model.mu

Initialize lists to store model parameter names, units and symbols

In [ ]:
params = []
units = []
symparams = []

**Known values of parameters that are not allowed to change (rhyolite-MELTS, 1.1, old water saturation model, new carbon dioxide model)**

In [ ]:
include_excess_parameters = False
if not include_excess_parameters:
    param_subst_l = []
known_w_values = [
                        26266.7, #   0 W(TiO2      ,SiO2      ) 0
                       -39120.0, #   1 W(Al2O3     ,SiO2      )
                         8110.3, #   2 W(Fe2O3     ,SiO2      )
                        27886.3, #   3 W(MgCr2O4   ,SiO2      )
                        23660.9, #   4 W(Fe2SiO4   ,SiO2      )
                        18393.9, #   5 W(MnSi0.5O2 ,SiO2      )
                         3421.0, #   6 W(Mg2SiO4   ,SiO2      )
                        25197.4, #   7 W(NiSi0.5O2 ,SiO2      )
                        14802.8, #   8 W(CoSi0.5O2 ,SiO2      )
                         -863.7, #   9 W(CaSiO3    ,SiO2      )
                       -99039.0, #  10 W(Na2SiO3   ,SiO2      ) 10
                       -33921.7, #  11 W(KAlSiO4   ,SiO2      )
                        61891.6, #  12 W(Ca3(PO4)2 ,SiO2      )
                        30967.3, #  13 W(H2O       ,SiO2      )
                            0.0, #  14 W(CO2       ,SiO2      )
     6.3281206517489e+01*1000.0, #  15 W(CaCO3     ,SiO2      )
                       -29449.8, #  16 W(Al2O3     ,TiO2      )
                       -84756.9, #  17 W(Fe2O3     ,TiO2      )
                       -72303.4, #  18 W(MgCr2O4   ,TiO2      )
                         5209.1, #  19 W(Fe2SiO4   ,TiO2      )
                       -16123.5, #  20 W(MnSi0.5O2 ,TiO2      ) 20
                        -4178.3, #  21 W(Mg2SiO4   ,TiO2      )
                         3614.8, #  22 W(NiSi0.5O2 ,TiO2      )
                        -1640.0, #  23 W(CoSi0.5O2 ,TiO2      )
                       -35372.5, #  24 W(CaSiO3    ,TiO2      )
                       -15415.6, #  25 W(Na2SiO3   ,TiO2      )
                       -48094.6, #  26 W(KAlSiO4   ,TiO2      )
                        25938.8, #  27 W(Ca3(PO4)2 ,TiO2      )
                        81879.1, #  28 W(H2O       ,TiO2      )
    -1.9265668647972e+01*1000.0, #  29 W(CO2       ,TiO2      )
    -7.9202707514741e+01*1000.0, #  30 W(CaCO3     ,TiO2      ) 30
                       -17089.4, #  31 W(Fe2O3     ,Al2O3     )
                       -31770.3, #  32 W(MgCr2O4   ,Al2O3     )
                       -30509.0, #  33 W(Fe2SiO4   ,Al2O3     )
                       -53874.9, #  34 W(MnSi0.5O2 ,Al2O3     )
                       -32880.3, #  35 W(Mg2SiO4   ,Al2O3     )
                         2985.2, #  36 W(NiSi0.5O2 ,Al2O3     )
                        -2677.4, #  37 W(CoSi0.5O2 ,Al2O3     )
                       -57917.9, #  38 7(CaSiO3    ,Al2O3     )
                      -130785.0, #  39 W(Na2SiO3   ,Al2O3     )
                       -25859.2, #  40 W(KAlSiO4   ,Al2O3     ) 40
                        52220.8, #  41 W(Ca3(PO4)2 ,Al2O3     )
                       -16098.1, #  42 W(H2O       ,Al2O3     )
                            0.0, #  43 W(CO2       ,Al2O3     )
     4.6716121152125e+01*1000.0, #  44 W(CaCO3     ,Al2O3     )
                        21605.9, #  45 W(MgCr2O4   ,Fe2O3     )
                      -179064.9, #  46 W(Fe2SiO4   ,Fe2O3     )
                         3907.9, #  47 W(MnSi0.5O2 ,Fe2O3     )
                       -71518.6, #  48 W(Mg2SiO4   ,Fe2O3     )
                          408.7, #  49 W(NiSi0.5O2 ,Fe2O3     )
                         -223.7, #  50 W(CoSi0.5O2 ,Fe2O3     ) 50
                        12076.6, #  51 W(CaSiO3    ,Fe2O3     )
                      -149662.2, #  52 W(Na2SiO3   ,Fe2O3     )
                        57555.9, #  53 W(KAlSiO4   ,Fe2O3     )
                        -4213.9, #  54 W(Ca3(PO4)2 ,Fe2O3     )
                        31405.5, #  55 W(H2O       ,Fe2O3     )
    -3.1868817914962e+00*1000.0, #  56 W(CO2       ,Fe2O3     )
     6.5508729974818e+01*1000.0, #  57 W(CaCO3     ,Fe2O3     )
                       -82971.8, #  58 W(Fe2SiO4   ,MgCr2O4   )
                          182.4, #  59 W(MnSi0.5O2 ,MgCr2O4   )
                        46049.2, #  60 W(Mg2SiO4   ,MgCr2O4   ) 60
                         -266.0, #  61 W(NiSi0.5O2 ,MgCr2O4   )
                         -384.0, #  62 W(CoSi0.5O2 ,MgCr2O4   )
                        30704.7, #  63 W(CaSiO3    ,MgCr2O4   )
                       113646.0, #  64 W(Na2SiO3   ,MgCr2O4   )
                        75709.1, #  65 W(KAlSiO4   ,MgCr2O4   )
                         5341.8, #  66 W(Ca3(PO4)2 ,MgCr2O4   )
                            0.0, #  67 W(H2O       ,MgCr2O4   )
                            0.0, #  68 W(CO2       ,MgCr2O4   )
                            0.0, #  69 W(CaCO3     ,MgCr2O4   )
                        -6823.9, #  70 W(MnSi0.5O2 ,Fe2SiO4   ) 70
                       -37256.7, #  71 W(Mg2SiO4   ,Fe2SiO4   )
                       -17019.8, #  72 W(NiSi0.5O2 ,Fe2SiO4   )
                       -11746.3, #  73 W(CoSi0.5O2 ,Fe2SiO4   )
                       -12970.8, #  74 W(CaSiO3    ,Fe2SiO4   )
                       -90533.8, #  75 W(Na2SiO3   ,Fe2SiO4   )
                        23649.4, #  76 W(KAlSiO4   ,Fe2SiO4   )
                        87410.3, #  77 W(Ca3(PO4)2 ,Fe2SiO4   )
                        28873.6, #  78 W(H2O       ,Fe2SiO4   )
    -3.2464545980393e+01*1000.0, #  79 W(CO2       ,Fe2SiO4   )
    -7.2996815311226e+01*1000.0, #  80 W(CaCO3     ,Fe2SiO4   ) 80
                       -13040.1, #  81 W(Mg2SiO4   ,MnSi0.5O2 )
                          785.8, #  82 W(NiSi0.5O2 ,MnSi0.5O2 )
                          -50.6, #  83 W(CoSi0.5O2 ,MnSi0.5O2 )
                         2934.6, #  84 W(CaSiO3    ,MnSi0.5O2 )
                       -15780.8, #  85 W(Na2SiO3   ,MnSi0.5O2 )
                        23727.4, #  86 W(KAlSiO4   ,MnSi0.5O2 )
                            0.0, #  87 W(Ca3(PO4)2 ,MnSi0.5O2 )
                            0.0, #  88 W(H2O       ,MnSi0.5O2 )
                            0.0, #  89 W(CO2       ,MnSi0.5O2 )
                            0.0, #  90 W(CaCO3     ,MnSi0.5O2 ) 90
                       -21175.5, #  91 W(NiSi0.5O2 ,Mg2SiO4   )
                       -14994.9, #  92 W(CoSi0.5O2 ,Mg2SiO4   )
                       -31731.9, #  93 W(CaSiO3    ,Mg2SiO4   )
                       -41876.9, #  94 W(Na2SiO3   ,Mg2SiO4   )
                        22323.1, #  95 W(KAlSiO4   ,Mg2SiO4   )
                       -23208.8, #  96 W(Ca3(PO4)2 ,Mg2SiO4   )
                        35633.7, #  97 W(H2O       ,Mg2SiO4   )
    -4.0853595138597e+01*1000.0, #  98 W(CO2       ,Mg2SiO4   )
    -2.4872562549739e+01*1000.0, #  99 W(CaCO3     ,Mg2SiO4   )
                          258.9, # 100 W(CoSi0.5O2 ,NiSi0.5O2 ) 100
                         7027.5, # 101 W(CaSiO3    ,NiSi0.5O2 )
                        -3647.8, # 102 W(Na2SiO3   ,NiSi0.5O2 )
                         4261.4, # 103 W(KAlSiO4   ,NiSi0.5O2 )
                            0.0, # 104 W(Ca3(PO4)2 ,NiSi0.5O2 )
                            0.0, # 105 W(H2O       ,NiSi0.5O2 )
                            0.0, # 106 W(CO2       ,NiSi0.5O2 )
                            0.0, # 107 W(CaCO3     ,NiSi0.5O2 )
                       -26685.7, # 108 W(CaSiO3    ,CoSi0.5O2 )
                          531.2, # 109 W(Na2SiO3   ,CoSi0.5O2 )
                          265.7, # 110 W(KAlSiO4   ,CoSi0.5O2 ) 110
                            0.0, # 111 W(Ca3(PO4)2 ,CoSi0.5O2 )
                            0.0, # 112 W(H2O       ,CoSi0.5O2 )
                            0.0, # 113 W(CO2       ,CoSi0.5O2 )
                            0.0, # 114 W(CaCO3     ,CoSi0.5O2 )
                       -13247.1, # 115 W(Na2SiO3   ,CaSiO3    )
                        17111.1, # 116 W(KAlSiO4   ,CaSiO3    )
                        37070.3, # 117 W(Ca3(PO4)2 ,CaSiO3    )
                        20374.6, # 118 W(H2O       ,CaSiO3    )
     3.0012481472120e+01*1000.0, # 119 W(CO2       ,CaSiO3    )
     3.7534127781795e+01*1000.0, # 120 W(CaCO3     ,CaSiO3    ) 120
                         6522.8, # 121 W(KAlSiO4   ,Na2SiO3   )
                        15571.9, # 122 W(Ca3(PO4)2 ,Na2SiO3   )
                       -96937.6, # 123 W(H2O       ,Na2SiO3   )
                            0.0, # 124 W(CO2       ,Na2SiO3   )
    -3.1101129932991e+02*1000.0, # 126 W(CaCO3     ,Na2SiO3   )
                        17100.6, # 127 W(Ca3(PO4)2 ,KAlSiO4   )
                        10374.2, # 128 W(H2O       ,KAlSiO4   )
                            0.0, # 129 W(CO2       ,KAlSiO4   )
    -2.7864967599360e+01*1000.0, # 130 W(CaCO3     ,KAlSiO4   )
                        43451.3, # 131 W(H2O       ,Ca3(PO4)2 ) 130
    -3.4728165916175e+00*1000.0, # 132 W(CO2       ,Ca3(PO4)2 )
     2.0119124588637e+00*1000.0, # 133 W(CaCO3     ,Ca3(PO4)2 )
     2.3255499914663e+01*1000.0, # 134 W(CO2       ,H2O       )
     7.8729683594722e+00*1000.0, # 135 W(CaCO3     ,H2O       )
                            0.0, # 180 W(CaCO3     ,CO2       )
]

## Define extensive standard state contribution to solution properties

In [ ]:
G_ss = sym.S.Zero
for i,x in enumerate(X_soln):
    if i < nc:
        G_ss += x*mu[i]
    else:
        h = sym.symbols('hd_'+str(i+1))
        params.append('hd_'+str(i+1))
        units.append('J')
        symparams.append(h)
        ss = sym.symbols('sd_'+str(i+1))
        symparams.append(ss)
        params.append('sd_'+str(i+1))
        units.append('J/K')
        v = sym.symbols('vd_'+str(i+1))
        params.append('vd_'+str(i+1))
        units.append('J/bar')
        symparams.append(v)
        G_ss += x*(h-T*ss+(P-sym.S.One)*v)
h = sym.symbols('hZrSiO4')
params.append('hZrSiO4')
units.append('J')
symparams.append(h)
ss = sym.symbols('sZrSiO4')
symparams.append(ss)
params.append('sZrSiO4')
units.append('J/K')
v = sym.symbols('vZrSiO4')
params.append('vZrSiO4')
units.append('J/bar')
symparams.append(v)
G_ss += X_soln[16]*(h-T*ss+(P-sym.S.One)*v)
G_ss

## Define extensive configurational Gibbs free energy

*R* is the gas constant.  
Note that species mole fractions are computed using *XT* rather than *nT*

In [ ]:
eps = np.finfo(float).eps

In [ ]:
R = sym.symbols('R')
S_config = sym.S.Zero
for x in X_soln:
    S_config += x*sym.log(x/XT+eps)
S_config += X_soln[14]*sym.log(X_soln[14]/XT) + (XT-X_soln[14])*sym.log(sym.S.One-X_soln[14]/XT) # extra entropy from hydroxyl term
S_config *= -R
G_config = -T*S_config

## Non-configurational extensive molar Gibbs free energy

These are the regular solution terms. Zirconium species are assumed to not contribute to the excess Gibbs free energy.  
The flag, *include_excess_parameters* is used to perform direct substitution of excess solution parameters, removing them from the list of “calibration” parameters in order to save code generation and compilation time.

In [ ]:
G_ex = sym.S.Zero
for i,x in enumerate(X_soln):
    istr = '{0:02d}'.format(i+1)
    for j,y in enumerate(X_soln):
        jstr = '{0:02d}'.format(j+1)
        if j > i and i<nc-1 and j<nc-1: #skip the last component, ZrSiO4
            w = sym.symbols('Wc'+istr+'c'+jstr)
            if include_excess_parameters:
                params.append('Wc'+istr+'c'+jstr)
                units.append('J')
                symparams.append(w)
            else:
                param_subst_l.append(w)
            G_ex += x*y*w
    if i<nc-1:
        w = sym.symbols('Wc'+istr+'s1')
        if include_excess_parameters:
            params.append('Wc'+istr+'s1')
            units.append('J')
            symparams.append(w)
        else:
            param_subst_l.append(w)
        G_ex += x*s[0]*w
G_ex /= XT

In [ ]:
print(params)
print(units)
print(symparams)

## Define the extensive Gibbs free energy of solution

In [ ]:
G = G_ss + G_config + G_ex

In [ ]:
ll = list(zip(param_subst_l,known_w_values))

In [ ]:
if not include_excess_parameters:
    G = G.subs(list(zip(param_subst_l,known_w_values)))

## Find the condition of homogeneous equilibrium: $\frac{{\partial \hat G}}{{\partial {s_i}}} = 0$

In [ ]:
dgds = []
for x in s:
    dgds.append(G.diff(x))

## Identify bounds on the ordering parameter

Construct a set of inequality constraints on the lower bound of species mole fractions. If all the mole fractions are guaranteed to be positive, then the sum of all those moles, *XT*, is positive. Note that the upper bound on *X<sub>i</sub>* is guaranteed to be feasible, because in use that quantity is always smaller than *XT*.

In [ ]:
ineq_l = [
    0 <= X_soln[ 0], 
    0 <= X_soln[ 1], 
    0 <= X_soln[ 2], 
    0 <= X_soln[ 3], 
    0 <= X_soln[ 4], 
    0 <= X_soln[ 5], 
    0 <= X_soln[ 6], 
    0 <= X_soln[ 7], 
    0 <= X_soln[ 8], 
    0 <= X_soln[ 9], 
    0 <= X_soln[10], 
    0 <= X_soln[11], 
    0 <= X_soln[12], 
    0 <= X_soln[13], 
    0 <= X_soln[14], 
    0 <= X_soln[15], 
    0 <= X_soln[16], 
    0 <= X_soln[17], 
    0 <= X_soln[18], 
    0 <= X_soln[19]
]
insym_l = [x for x in n]
out = sym.reduce_inequalities(inequalities=ineq_l, symbols=insym_l)
out

## Identity initial guess values of the order parameters

In [ ]:
initial_guess_l = []
initial_guess_l.append( sym.Min(n[10]/6,n[15]/2) )
initial_guess_l.append( sym.Min(n[11]/2,n[16]/4) )
initial_guess_l.append( sym.Min(n[12]/4,n[16]/4) )

In [ ]:
initial_guess_l

## Identify null values of ordering parameters

In [ ]:
nullValues = [sym.S.Zero, sym.S.Zero, sym.S.Zero]

## Add the Gibbs free energy of solution to the model

In [ ]:
model.add_expression_to_model(G, list(zip(params, units, symparams)), ordering_functions=(dgds,s,initial_guess_l,out,nullValues))

## Name the module that will hold the liquid model

In [ ]:
model.module = "rMELTS_ZR"

## Composition conversion and bound constraints

Develop expressions that convert solution composition expressed as moles of elements into moles of components

In [ ]:
ineq_l = []
insym_l = []
for key,value in elm.items():
    y = sym.symbols(key)
    ineq_l.append(value - y)
    insym_l.append(y)
sol = sym.solve(ineq_l, tuple(n))
for i,x in enumerate(n):
    print ('n['+str(i+1)+'] = ', end='')
    print(sol[x])

Construct a formula string for the model

In [ ]:
repl_s = {}
for key in elm.keys():
    if key[0:1] != ' ':
        repl_s[key] = '[' + key + ']'
for key in elm.keys():
    if key[0:1] == ' ':
        repl_s[key[1:]] = '[' + key[1] + ']'
formula_string = ''
for x,y in repl_s.items():
    formula_string += x + y
formula_string

Compute a list of conversion strings to cast element abundances to component mole numbers

In [ ]:
def process_str(z):
    while True:
        l1 = z.find('[[')
        if l1 == -1:
            return z
        else:
            l2 = l1+3
            z = z[:l2] + z[l2+1:]
            z = z[:l1] + z[l1+1:]

In [ ]:
conversion_string = []
for i,zz in enumerate(sol):
    z = str(sol[zz]).replace(' ', '')
    for x,y in repl_s.items():
        z = z.replace(x,y)
    z = process_str(z)
    conversion_string.append('['+str(i)+']='+z)
conversion_string

Compute a test string to insure a feasible solution; these constraints insure that the moles of each element is greater than zero

In [ ]:
test_string = []
for x,y in elm.items():
    z = str(y).replace('/2 +',']/2+')
    z = z.replace(' +', '] +')
    z = z.replace('n', '[')
    z = z.replace(' ', '')
    z += '] >= 0.0'
    z = z.replace('/2]',']/2')
    for i in range(1,nc+1):
        z = z.replace('['+str(i)+']', '['+str(i-1)+']')
    test_string.append(z)

## Add these bound constraints to the model

In [ ]:
model.formula_string = formula_string
model.conversion_string = conversion_string
model.test_string = test_string

## Assign parameter values

In [ ]:
depen_species_values = [
    0.0, 0.0, 0.0, # H, S, V, CaCO3
    0.0, 0.0, 0.0, # H, S, V, Na4ZrSi2O8
    0.0, 0.0, 0.0, # H, S, V, K4ZrSi2O8
    0.0, 0.0, 0.0  # H. S. V, ZrSiO4
]

Parameter values are consistent with rhyolite-MELTS version 1.1

In [ ]:
if include_excess_parameters:
    w_values = [
                        26266.7, #   0 W(TiO2      ,SiO2      ) 0
                       -39120.0, #   1 W(Al2O3     ,SiO2      )
                         8110.3, #   2 W(Fe2O3     ,SiO2      )
                        27886.3, #   3 W(MgCr2O4   ,SiO2      )
                        23660.9, #   4 W(Fe2SiO4   ,SiO2      )
                        18393.9, #   5 W(MnSi0.5O2 ,SiO2      )
                         3421.0, #   6 W(Mg2SiO4   ,SiO2      )
                        25197.4, #   7 W(NiSi0.5O2 ,SiO2      )
                        14802.8, #   8 W(CoSi0.5O2 ,SiO2      )
                         -863.7, #   9 W(CaSiO3    ,SiO2      )
                       -99039.0, #  10 W(Na2SiO3   ,SiO2      ) 10
                       -33921.7, #  11 W(KAlSiO4   ,SiO2      )
                        61891.6, #  12 W(Ca3(PO4)2 ,SiO2      )
                        30967.3, #  13 W(H2O       ,SiO2      )
                            0.0, #  14 W(CO2       ,SiO2      )
     6.3281206517489e+01*1000.0, #  15 W(CaCO3     ,SiO2      )
                       -29449.8, #  16 W(Al2O3     ,TiO2      )
                       -84756.9, #  17 W(Fe2O3     ,TiO2      )
                       -72303.4, #  18 W(MgCr2O4   ,TiO2      )
                         5209.1, #  19 W(Fe2SiO4   ,TiO2      )
                       -16123.5, #  20 W(MnSi0.5O2 ,TiO2      ) 20
                        -4178.3, #  21 W(Mg2SiO4   ,TiO2      )
                         3614.8, #  22 W(NiSi0.5O2 ,TiO2      )
                        -1640.0, #  23 W(CoSi0.5O2 ,TiO2      )
                       -35372.5, #  24 W(CaSiO3    ,TiO2      )
                       -15415.6, #  25 W(Na2SiO3   ,TiO2      )
                       -48094.6, #  26 W(KAlSiO4   ,TiO2      )
                        25938.8, #  27 W(Ca3(PO4)2 ,TiO2      )
                        81879.1, #  28 W(H2O       ,TiO2      )
    -1.9265668647972e+01*1000.0, #  29 W(CO2       ,TiO2      )
    -7.9202707514741e+01*1000.0, #  30 W(CaCO3     ,TiO2      ) 30
                       -17089.4, #  31 W(Fe2O3     ,Al2O3     )
                       -31770.3, #  32 W(MgCr2O4   ,Al2O3     )
                       -30509.0, #  33 W(Fe2SiO4   ,Al2O3     )
                       -53874.9, #  34 W(MnSi0.5O2 ,Al2O3     )
                       -32880.3, #  35 W(Mg2SiO4   ,Al2O3     )
                         2985.2, #  36 W(NiSi0.5O2 ,Al2O3     )
                        -2677.4, #  37 W(CoSi0.5O2 ,Al2O3     )
                       -57917.9, #  38 7(CaSiO3    ,Al2O3     )
                      -130785.0, #  39 W(Na2SiO3   ,Al2O3     )
                       -25859.2, #  40 W(KAlSiO4   ,Al2O3     ) 40
                        52220.8, #  41 W(Ca3(PO4)2 ,Al2O3     )
                       -16098.1, #  42 W(H2O       ,Al2O3     )
                            0.0, #  43 W(CO2       ,Al2O3     )
     4.6716121152125e+01*1000.0, #  44 W(CaCO3     ,Al2O3     )
                        21605.9, #  45 W(MgCr2O4   ,Fe2O3     )
                      -179064.9, #  46 W(Fe2SiO4   ,Fe2O3     )
                         3907.9, #  47 W(MnSi0.5O2 ,Fe2O3     )
                       -71518.6, #  48 W(Mg2SiO4   ,Fe2O3     )
                          408.7, #  49 W(NiSi0.5O2 ,Fe2O3     )
                         -223.7, #  50 W(CoSi0.5O2 ,Fe2O3     ) 50
                        12076.6, #  51 W(CaSiO3    ,Fe2O3     )
                      -149662.2, #  52 W(Na2SiO3   ,Fe2O3     )
                        57555.9, #  53 W(KAlSiO4   ,Fe2O3     )
                        -4213.9, #  54 W(Ca3(PO4)2 ,Fe2O3     )
                        31405.5, #  55 W(H2O       ,Fe2O3     )
    -3.1868817914962e+00*1000.0, #  56 W(CO2       ,Fe2O3     )
     6.5508729974818e+01*1000.0, #  57 W(CaCO3     ,Fe2O3     )
                       -82971.8, #  58 W(Fe2SiO4   ,MgCr2O4   )
                          182.4, #  59 W(MnSi0.5O2 ,MgCr2O4   )
                        46049.2, #  60 W(Mg2SiO4   ,MgCr2O4   ) 60
                         -266.0, #  61 W(NiSi0.5O2 ,MgCr2O4   )
                         -384.0, #  62 W(CoSi0.5O2 ,MgCr2O4   )
                        30704.7, #  63 W(CaSiO3    ,MgCr2O4   )
                       113646.0, #  64 W(Na2SiO3   ,MgCr2O4   )
                        75709.1, #  65 W(KAlSiO4   ,MgCr2O4   )
                         5341.8, #  66 W(Ca3(PO4)2 ,MgCr2O4   )
                            0.0, #  67 W(H2O       ,MgCr2O4   )
                            0.0, #  68 W(CO2       ,MgCr2O4   )
                            0.0, #  69 W(CaCO3     ,MgCr2O4   )
                        -6823.9, #  70 W(MnSi0.5O2 ,Fe2SiO4   ) 70
                       -37256.7, #  71 W(Mg2SiO4   ,Fe2SiO4   )
                       -17019.8, #  72 W(NiSi0.5O2 ,Fe2SiO4   )
                       -11746.3, #  73 W(CoSi0.5O2 ,Fe2SiO4   )
                       -12970.8, #  74 W(CaSiO3    ,Fe2SiO4   )
                       -90533.8, #  75 W(Na2SiO3   ,Fe2SiO4   )
                        23649.4, #  76 W(KAlSiO4   ,Fe2SiO4   )
                        87410.3, #  77 W(Ca3(PO4)2 ,Fe2SiO4   )
                        28873.6, #  78 W(H2O       ,Fe2SiO4   )
    -3.2464545980393e+01*1000.0, #  79 W(CO2       ,Fe2SiO4   )
    -7.2996815311226e+01*1000.0, #  80 W(CaCO3     ,Fe2SiO4   ) 80
                       -13040.1, #  81 W(Mg2SiO4   ,MnSi0.5O2 )
                          785.8, #  82 W(NiSi0.5O2 ,MnSi0.5O2 )
                          -50.6, #  83 W(CoSi0.5O2 ,MnSi0.5O2 )
                         2934.6, #  84 W(CaSiO3    ,MnSi0.5O2 )
                       -15780.8, #  85 W(Na2SiO3   ,MnSi0.5O2 )
                        23727.4, #  86 W(KAlSiO4   ,MnSi0.5O2 )
                            0.0, #  87 W(Ca3(PO4)2 ,MnSi0.5O2 )
                            0.0, #  88 W(H2O       ,MnSi0.5O2 )
                            0.0, #  89 W(CO2       ,MnSi0.5O2 )
                            0.0, #  90 W(CaCO3     ,MnSi0.5O2 ) 90
                       -21175.5, #  91 W(NiSi0.5O2 ,Mg2SiO4   )
                       -14994.9, #  92 W(CoSi0.5O2 ,Mg2SiO4   )
                       -31731.9, #  93 W(CaSiO3    ,Mg2SiO4   )
                       -41876.9, #  94 W(Na2SiO3   ,Mg2SiO4   )
                        22323.1, #  95 W(KAlSiO4   ,Mg2SiO4   )
                       -23208.8, #  96 W(Ca3(PO4)2 ,Mg2SiO4   )
                        35633.7, #  97 W(H2O       ,Mg2SiO4   )
    -4.0853595138597e+01*1000.0, #  98 W(CO2       ,Mg2SiO4   )
    -2.4872562549739e+01*1000.0, #  99 W(CaCO3     ,Mg2SiO4   )
                          258.9, # 100 W(CoSi0.5O2 ,NiSi0.5O2 ) 100
                         7027.5, # 101 W(CaSiO3    ,NiSi0.5O2 )
                        -3647.8, # 102 W(Na2SiO3   ,NiSi0.5O2 )
                         4261.4, # 103 W(KAlSiO4   ,NiSi0.5O2 )
                            0.0, # 104 W(Ca3(PO4)2 ,NiSi0.5O2 )
                            0.0, # 105 W(H2O       ,NiSi0.5O2 )
                            0.0, # 106 W(CO2       ,NiSi0.5O2 )
                            0.0, # 107 W(CaCO3     ,NiSi0.5O2 )
                       -26685.7, # 108 W(CaSiO3    ,CoSi0.5O2 )
                          531.2, # 109 W(Na2SiO3   ,CoSi0.5O2 )
                          265.7, # 110 W(KAlSiO4   ,CoSi0.5O2 ) 110
                            0.0, # 111 W(Ca3(PO4)2 ,CoSi0.5O2 )
                            0.0, # 112 W(H2O       ,CoSi0.5O2 )
                            0.0, # 113 W(CO2       ,CoSi0.5O2 )
                            0.0, # 114 W(CaCO3     ,CoSi0.5O2 )
                       -13247.1, # 115 W(Na2SiO3   ,CaSiO3    )
                        17111.1, # 116 W(KAlSiO4   ,CaSiO3    )
                        37070.3, # 117 W(Ca3(PO4)2 ,CaSiO3    )
                        20374.6, # 118 W(H2O       ,CaSiO3    )
     3.0012481472120e+01*1000.0, # 119 W(CO2       ,CaSiO3    )
     3.7534127781795e+01*1000.0, # 120 W(CaCO3     ,CaSiO3    ) 120
                         6522.8, # 121 W(KAlSiO4   ,Na2SiO3   )
                        15571.9, # 122 W(Ca3(PO4)2 ,Na2SiO3   )
                       -96937.6, # 123 W(H2O       ,Na2SiO3   )
                            0.0, # 124 W(CO2       ,Na2SiO3   )
    -3.1101129932991e+02*1000.0, # 126 W(CaCO3     ,Na2SiO3   )
                        17100.6, # 127 W(Ca3(PO4)2 ,KAlSiO4   )
                        10374.2, # 128 W(H2O       ,KAlSiO4   )
                            0.0, # 129 W(CO2       ,KAlSiO4   )
    -2.7864967599360e+01*1000.0, # 130 W(CaCO3     ,KAlSiO4   )
                        43451.3, # 131 W(H2O       ,Ca3(PO4)2 ) 130
    -3.4728165916175e+00*1000.0, # 132 W(CO2       ,Ca3(PO4)2 )
     2.0119124588637e+00*1000.0, # 133 W(CaCO3     ,Ca3(PO4)2 )
     2.3255499914663e+01*1000.0, # 134 W(CO2       ,H2O       )
     7.8729683594722e+00*1000.0, # 135 W(CaCO3     ,H2O       )
                            0.0, # 180 W(CaCO3     ,CO2       )
    ]
else:
    w_values = []

In [ ]:
paramValues = dict(zip(params, depen_species_values+w_values))

In [ ]:
paramValues['T_r'] = 298.15
paramValues['P_r'] =   1.0

## Generate code, load and compile the module

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}
model_type = "calib"

In [ ]:
model.verbose = True
model.de_minimis = True
model.create_code_module(phase="Liquid", 
                         params=paramValues, 
                         endmembers=[
                             'SiO2_rMELTS_ZR',
                             'TiO2_rMELTS_ZR',
                             'Al2O3_rMELTS_ZR',
                             'Fe2O3_rMELTS_ZR',
                             'MgCr2O4_rMELTS_ZR',
                             'Fe2SiO4_rMELTS_ZR',
                             'MnSi1d2O2_rMELTS_ZR',
                             'Mg2SiO4_rMELTS_ZR',
                             'NiSi1d2O2_rMELTS_ZR',
                             'CoSi1d2O2_rMELTS_ZR',
                             'CaSiO3_rMELTS_ZR',
                             'Na2SiO3_rMELTS_ZR',
                             'KAlSiO4_rMELTS_ZR',
                             'Ca3P2O8_rMELTS_ZR',
                             'H2O_rMELTS_ZR',
                             'CO2_rMELTS_ZR',
                             'ZrSiO4_rMELTS_ZR'
                         ], 
                         prefix="cy", 
                         module_type=model_type, 
                         silent=False, 
                         add_code_to_access_order_paramater=True,
                         minimal_deriv_set=True)

In [ ]:
import rMELTS_ZR
%cd ..

## Test identifier routines

In [ ]:
print(rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_identifier())
print(rMELTS_ZR.cy_Liquid_rMELTS_ZR_calib_name())